In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

## Reading data

In [2]:
dados = pd.read_csv('DATA.txt', sep="   ", header=None, engine='python')
dados.columns = ["gender", "age", "risk_factors", "systolic_bp", "s_hr", "st_segment", "ecg_hr", "creatinine", "killp", "event"]
display(dados)

,gender,age,risk_factors,systolic_bp,s_hr,st_segment,ecg_hr,creatinine,killp,event
0,1.0,33.0,0.0,132.0,92.266205,1.0,90.0,0.8,1.0,0.0
1,1.0,69.0,0.0,147.0,54.178624,0.0,52.0,1.4,1.0,0.0
2,1.0,63.0,0.0,142.0,41.364843,1.0,44.0,1.1,3.0,1.0
3,0.0,79.0,0.0,147.0,107.379000,1.0,110.0,0.9,1.0,1.0
4,0.0,61.0,0.0,107.0,83.224808,0.0,80.0,1.1,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
452,1.0,69.0,0.0,95.0,100.960080,0.0,103.0,1.6,3.0,1.0
453,1.0,51.0,1.0,140.0,87.259367,1.0,90.0,1.0,3.0,1.0
454,0.0,57.0,0.0,120.0,114.649890,0.0,117.0,1.0,3.0,1.0
455,1.0,87.0,0.0,149.0,48.788823,1.0,53.0,1.4,1.0,1.0


## Pre-processing of data

In [3]:
# Converts the data on a given column from float to integer
def float_to_int(data, column):
    data[column] = data[column].astype(int)

In [4]:
data = dados.copy()

# Data pre-processing (floats to integers)
float_to_int(data, "gender")
float_to_int(data, "risk_factors")
float_to_int(data, "st_segment")
float_to_int(data, "killp")
float_to_int(data, "event")

display(data)

,gender,age,risk_factors,systolic_bp,s_hr,st_segment,ecg_hr,creatinine,killp,event
0,1,33.0,0,132.0,92.266205,1,90.0,0.8,1,0
1,1,69.0,0,147.0,54.178624,0,52.0,1.4,1,0
2,1,63.0,0,142.0,41.364843,1,44.0,1.1,3,1
3,0,79.0,0,147.0,107.379000,1,110.0,0.9,1,1
4,0,61.0,0,107.0,83.224808,0,80.0,1.1,1,0
...,...,...,...,...,...,...,...,...,...,...
452,1,69.0,0,95.0,100.960080,0,103.0,1.6,3,1
453,1,51.0,1,140.0,87.259367,1,90.0,1.0,3,1
454,0,57.0,0,120.0,114.649890,0,117.0,1.0,3,1
455,1,87.0,0,149.0,48.788823,1,53.0,1.4,1,1


## Fusion of HR data of ECG and device

In [5]:
# Fusion of the same information of 2 different devices
def fuse_same_sensors_mult(df):
    dp = np.array([2, 0.5])
    tau = np.sum(1 / np.power(dp, 2))
    s = 1 / tau
    
    valores = np.zeros(len(dados.s_hr))
    for i in range(len(dados["s_hr"])):
        valores[i] = (dados['s_hr'][i] / (tau * np.power(dp[0], 2))) + (dados['ecg_hr'][i] / (tau * np.power(dp[1], 2)))

    df["hr_final"] = valores

In [6]:
fuse_same_sensors_mult(data)

display(data)

,gender,age,risk_factors,systolic_bp,s_hr,st_segment,ecg_hr,creatinine,killp,event,hr_final
0,1,33.0,0,132.0,92.266205,1,90.0,0.8,1,0,90.133306
1,1,69.0,0,147.0,54.178624,0,52.0,1.4,1,0,52.128154
2,1,63.0,0,142.0,41.364843,1,44.0,1.1,3,1,43.844991
3,0,79.0,0,147.0,107.379000,1,110.0,0.9,1,1,109.845824
4,0,61.0,0,107.0,83.224808,0,80.0,1.1,1,0,80.189695
...,...,...,...,...,...,...,...,...,...,...,...
452,1,69.0,0,95.0,100.960080,0,103.0,1.6,3,1,102.880005
453,1,51.0,1,140.0,87.259367,1,90.0,1.0,3,1,89.838786
454,0,57.0,0,120.0,114.649890,0,117.0,1.0,3,1,116.861758
455,1,87.0,0,149.0,48.788823,1,53.0,1.4,1,1,52.752284


## Clinical guidelines
- Creatinine levels >= 1.2
- Have the ST segment

In [7]:
data['clinical'] = np.where((data["creatinine"] >= 1.2) & (data["st_segment"] == 1), 1, 0)

display(data)

,gender,age,risk_factors,systolic_bp,s_hr,st_segment,ecg_hr,creatinine,killp,event,hr_final,clinical
0,1,33.0,0,132.0,92.266205,1,90.0,0.8,1,0,90.133306,0
1,1,69.0,0,147.0,54.178624,0,52.0,1.4,1,0,52.128154,0
2,1,63.0,0,142.0,41.364843,1,44.0,1.1,3,1,43.844991,0
3,0,79.0,0,147.0,107.379000,1,110.0,0.9,1,1,109.845824,0
4,0,61.0,0,107.0,83.224808,0,80.0,1.1,1,0,80.189695,0
...,...,...,...,...,...,...,...,...,...,...,...,...
452,1,69.0,0,95.0,100.960080,0,103.0,1.6,3,1,102.880005,0
453,1,51.0,1,140.0,87.259367,1,90.0,1.0,3,1,89.838786,0
454,0,57.0,0,120.0,114.649890,0,117.0,1.0,3,1,116.861758,0
455,1,87.0,0,149.0,48.788823,1,53.0,1.4,1,1,52.752284,1


## Continuous Data
- age
- systolic_bp
- hr_final (s_hr + ecg_hr)
- creatinine

In [8]:
# Calculates means and std values of given columns
def means_stds(df, columns, target):
    means_std = {}

    mean_0 = []
    std_0 = []
    mean_1 = []
    std_1 = []
    
    for i in columns:
        mean_0.append(df[df[target] == 0][i].mean())
        mean_1.append(df[df[target] == 1][i].mean())

        std_0.append(df[df[target] == 0][i].std())
        std_1.append(df[df[target] == 1][i].std())

    means_std["mean_0"] = mean_0
    means_std["mean_1"] = mean_1

    means_std["std_0"] = std_0
    means_std["std_1"] = std_1
    
    return means_std


# Returns the probability distribution function value given mean, std and x value
def normpdf(x, mean, std):
    return norm.pdf(x, mean, std)


# Makes the fusion of the continuous data
def continuous_data(x, mean, std):
    aux = []
    
    for i in range(len(x)):
        aux.append(normpdf(x[i], mean[i], std[i]))
        
    return np.prod(aux)

In [9]:
columns = ["creatinine", "age", "systolic_bp", "hr_final"]

pred_continuous = np.zeros(len(data))
finals_continuous = np.zeros((len(data), 2))

mean_std = means_stds(data, columns, "event")

for i in range(len(data)):
    x = np.array([data.loc[i]["creatinine"], data.loc[i]["age"], data.loc[i]["systolic_bp"], data.loc[i]["hr_final"]])

    cont_dont = continuous_data(x, mean_std["mean_0"], mean_std["std_0"])
    cont_have = continuous_data(x, mean_std["mean_1"], mean_std["std_1"])

    finals_continuous[i][0] = cont_dont
    finals_continuous[i][1] = cont_have
    
    if cont_have > cont_dont:
        pred_continuous[i] = 1
    else:
        pred_continuous[i] = 0
        
# Create column on data with the predictions only using continuous data
data["pred_cont"] = pred_continuous.astype(int)
display(data)

,gender,age,risk_factors,systolic_bp,s_hr,st_segment,ecg_hr,creatinine,killp,event,hr_final,clinical,pred_cont
0,1,33.0,0,132.0,92.266205,1,90.0,0.8,1,0,90.133306,0,0
1,1,69.0,0,147.0,54.178624,0,52.0,1.4,1,0,52.128154,0,0
2,1,63.0,0,142.0,41.364843,1,44.0,1.1,3,1,43.844991,0,0
3,0,79.0,0,147.0,107.379000,1,110.0,0.9,1,1,109.845824,0,1
4,0,61.0,0,107.0,83.224808,0,80.0,1.1,1,0,80.189695,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,1,69.0,0,95.0,100.960080,0,103.0,1.6,3,1,102.880005,0,1
453,1,51.0,1,140.0,87.259367,1,90.0,1.0,3,1,89.838786,0,0
454,0,57.0,0,120.0,114.649890,0,117.0,1.0,3,1,116.861758,0,1
455,1,87.0,0,149.0,48.788823,1,53.0,1.4,1,1,52.752284,1,1


## Discrete Data
- gender
- risk_factors
- st_segment
- killp

In [10]:
# Calculates the likelihood of the x values (conditional probabilities)
def likelihood(x, df, columns, target):
    res = {}
    uniques = df[target].unique()
    for i in range(len(uniques)):
        calc = np.zeros(len(columns))
        for j in range(len(columns)):            
            calc[j] = len(df[(df[target] == uniques[i]) & (df[columns[j]] == x[j])]) / len(df[df[target] == uniques[i]])
        res[uniques[i]] = np.prod(calc)
    return res

In [11]:
p_event_0 = len(data[data["event"] == 0]) / len(data)
p_event_1 = len(data[data["event"] == 1]) / len(data)

columns_disc = ["gender", "risk_factors", "st_segment", "killp", "clinical"]

finals_discrete = np.zeros((len(data), 2))

pred_discrete = np.zeros(len(data))

for i in range(len(data)):
    x_disc = np.array([data.loc[i]["gender"], data.loc[i]["risk_factors"], data.loc[i]["st_segment"], data.loc[i]["killp"], data.loc[i]["clinical"]])

    likelihoods = likelihood(x_disc, data, columns_disc, "event")
    
    final_0 = likelihoods[0] * p_event_0
    final_1 = likelihoods[1] * p_event_1
    
    finals_discrete[i][0] = final_0
    finals_discrete[i][1] = final_1
    
    if final_1 > final_0:
        pred_discrete[i] = 1
    else:
        pred_discrete[i] = 0
        
# Create column on data with the predictions only using discrete data
data["pred_disc"] = pred_discrete.astype(int)
display(data)

,gender,age,risk_factors,systolic_bp,s_hr,st_segment,ecg_hr,creatinine,killp,event,hr_final,clinical,pred_cont,pred_disc
0,1,33.0,0,132.0,92.266205,1,90.0,0.8,1,0,90.133306,0,0,0
1,1,69.0,0,147.0,54.178624,0,52.0,1.4,1,0,52.128154,0,0,0
2,1,63.0,0,142.0,41.364843,1,44.0,1.1,3,1,43.844991,0,0,1
3,0,79.0,0,147.0,107.379000,1,110.0,0.9,1,1,109.845824,0,1,0
4,0,61.0,0,107.0,83.224808,0,80.0,1.1,1,0,80.189695,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,1,69.0,0,95.0,100.960080,0,103.0,1.6,3,1,102.880005,0,1,1
453,1,51.0,1,140.0,87.259367,1,90.0,1.0,3,1,89.838786,0,0,1
454,0,57.0,0,120.0,114.649890,0,117.0,1.0,3,1,116.861758,0,1,1
455,1,87.0,0,149.0,48.788823,1,53.0,1.4,1,1,52.752284,1,1,1


## Predicted values and comparison

In [12]:
# Calculates the geometric mean
def geo_mean(iterable):
    a = np.array(iterable)
    return a.prod() ** (1.0 / len(a))

In [13]:
# Fusion of continuous and discrete data
final = finals_discrete * finals_continuous

# Create new column with the prediction of the previous fusion
data["pred_final"] = (final[:, 0] < final[:, 1]).astype(int)
display(data)

,gender,age,risk_factors,systolic_bp,s_hr,st_segment,ecg_hr,creatinine,killp,event,hr_final,clinical,pred_cont,pred_disc,pred_final
0,1,33.0,0,132.0,92.266205,1,90.0,0.8,1,0,90.133306,0,0,0,0
1,1,69.0,0,147.0,54.178624,0,52.0,1.4,1,0,52.128154,0,0,0,0
2,1,63.0,0,142.0,41.364843,1,44.0,1.1,3,1,43.844991,0,0,1,1
3,0,79.0,0,147.0,107.379000,1,110.0,0.9,1,1,109.845824,0,1,0,1
4,0,61.0,0,107.0,83.224808,0,80.0,1.1,1,0,80.189695,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,1,69.0,0,95.0,100.960080,0,103.0,1.6,3,1,102.880005,0,1,1,1
453,1,51.0,1,140.0,87.259367,1,90.0,1.0,3,1,89.838786,0,0,1,1
454,0,57.0,0,120.0,114.649890,0,117.0,1.0,3,1,116.861758,0,1,1,1
455,1,87.0,0,149.0,48.788823,1,53.0,1.4,1,1,52.752284,1,1,1,1


In [14]:
# Real Data / Predicted Data
display(data[["event", "pred_final"]])

,event,pred_final
0,0,0
1,0,0
2,1,1
3,1,1
4,0,0
...,...,...
452,1,1
453,1,1
454,1,1
455,1,1


## Analysis of the results

In [15]:
tn, fp, fn, tp = confusion_matrix(data["event"], data["pred_final"]).ravel()

# SENSITIVITY: the ability of a test to correctly identify those with the disease (true positive rate),
sensitivity = tp / (tp + fn)

# SPECIFICITY: the ability of the test to correctly identify those without the disease (true negative rate)
specificity = tn / (tn + fp)

### Confusion Matrix (with the following format)
 <b>TN</b>    <b>FP</b><br>
 <b>FN</b>    <b>TP</b>

In [16]:
# Confusion Matrix
print(confusion_matrix(data["event"], data["pred_disc"]))

[[245  33]
 [ 72 107]]


- Sensitivity
- Specificity
- Arithmetic Average of Sensitivity and Specificity
- Geometric Average of Sensitivity and Specificity
- Accuracy

In [17]:
print(f'Sensitivity: {np.round(sensitivity, 5)}\nSpecificity: {np.round(specificity, 5)}\n')
print(f'Arithmetic Average: {np.round(specificity * sensitivity, 5)}\nGeometric Average: {np.round(geo_mean([specificity, sensitivity]), 5)}\n')
print(f'Accuracy: {np.round(accuracy_score(data["event"], data["pred_final"]), 5)}')

Sensitivity: 0.69274
Specificity: 0.90647

Arithmetic Average: 0.62795
Geometric Average: 0.79243

Accuracy: 0.82276
